# Aviation Data

In [1]:
import os
os.chdir("../../")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import itertools

In [2]:
ad = pd.read_excel(os.getcwd() +"/data/tourism/aviation_seats_flights_pic.xlsx")
ad.columns = [col.lower() for col in ad.columns]
ad["date"] = pd.to_datetime(ad["date"])
ad.head(5)

,country,iso,region,date,aircraft_type,seats_arrivals_domestic,seats_arrivals_interregional,seats_arrivals_intraregional,seats_arrivals_intl,seats_arrivals_total,available_seat_kilometers,number_of_flights_domestic,number_of_flights_interregional,number_of_flights_intraregional,number_of_flights_intl,number_of_flights_total
0,Fiji,FJ,East Asia & Pacific,2019-01-01,passenger,839,273,3480,3753,4592,1.430416e+07,8,1,10,11,19
1,Fiji,FJ,East Asia & Pacific,2019-01-02,passenger,974,313,3471,3784,4758,1.495610e+07,8,1,10,11,19
2,Fiji,FJ,East Asia & Pacific,2019-01-03,passenger,1190,443,3675,4118,5308,1.592143e+07,10,2,12,14,24
3,Fiji,FJ,East Asia & Pacific,2019-01-04,passenger,831,586,3159,3745,4576,1.457334e+07,7,2,12,14,21
4,Fiji,FJ,East Asia & Pacific,2019-01-05,passenger,744,273,4752,5025,5769,1.773449e+07,7,1,12,13,20


In [3]:
ad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10611 entries, 0 to 10610
Data columns (total 16 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   country                          10611 non-null  object        
 1   iso                              10611 non-null  object        
 2   region                           10611 non-null  object        
 3   date                             10611 non-null  datetime64[ns]
 4   aircraft_type                    10566 non-null  object        
 5   seats_arrivals_domestic          10611 non-null  int64         
 6   seats_arrivals_interregional     10611 non-null  int64         
 7   seats_arrivals_intraregional     10611 non-null  int64         
 8   seats_arrivals_intl              10611 non-null  int64         
 9   seats_arrivals_total             10611 non-null  int64         
 10  available_seat_kilometers        10611 non-null  float64  

In [4]:
select_cols = ["country", "date", "seats_arrivals_intl", "seats_arrivals_total",
               "number_of_flights_intl", "number_of_flights_total"]

# Subset to the passenger
ad = (ad[ad["aircraft_type"] == "passenger"][select_cols]
      .reset_index().drop("index", axis=1))
ad.head(5)

,country,date,seats_arrivals_intl,seats_arrivals_total,number_of_flights_intl,number_of_flights_total
0,Fiji,2019-01-01,3753,4592,11,19
1,Fiji,2019-01-02,3784,4758,11,19
2,Fiji,2019-01-03,4118,5308,14,24
3,Fiji,2019-01-04,3745,4576,14,21
4,Fiji,2019-01-05,5025,5769,13,20


In [6]:
daterange = pd.date_range(start=ad.date.min(),
                          end=ad.date.max(),
                          freq="D").tolist()
countries = ad.country.unique().tolist()

comb = itertools.product(daterange, countries)

In [7]:
date_df = pd.DataFrame()
for i in comb:
    row = pd.DataFrame(i).T
    date_df = pd.concat([date_df, row], axis=0)
date_df.columns = ["date", "country"]

ad = date_df.merge(ad, how="left").fillna(0)
ad.sample(5)

,date,country,seats_arrivals_intl,seats_arrivals_total,number_of_flights_intl,number_of_flights_total
13599,2022-02-07,Micronesia (Federated States of),0.0,0.0,0.0,0.0
14149,2022-03-25,Kiribati,0.0,0.0,0.0,0.0
15179,2022-06-18,Vanuatu,0.0,0.0,0.0,0.0
3658,2019-11-01,Tuvalu,0.0,0.0,0.0,0.0
7313,2020-09-01,Palau,0.0,0.0,0.0,0.0
